- เพิ่มงานใหม่: ผู้ใช้สามารถเพิ่มงานโดยมีชื่อ, คำอธิบาย, และวันที่ครบกำหนด โดยแต่ละงานควรมีรหัสเฉพาะ (ID) ของตัวเอง
- ดูงานทั้งหมด: แสดงงานทั้งหมดในรูปแบบที่มีโครงสร้าง โดยแยกงานออกเป็นประเภท "รอดำเนินการ" และ "เสร็จสิ้น"
- ทำเครื่องหมายว่างานเสร็จสิ้น: มีฟังก์ชันให้ผู้ใช้ทำเครื่องหมายว่างานเสร็จสิ้น โดยอ้างอิงจากรหัสเฉพาะ (ID) ของงาน
- ลบงาน: ผู้ใช้สามารถลบงานโดยใช้รหัสเฉพาะ (ID)
- บันทึกและโหลดงาน: งานทั้งหมดควรถูกบันทึกลงในไฟล์ JSON และโหลดกลับเมื่อโปรแกรมเริ่มต้น
- ค้นหางาน: เพิ่มฟังก์ชันสำหรับการค้นหางานโดยใช้คำสำคัญหรือวันที่ครบกำหนด